In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from notebooks.ford.vins import load_vins

vins = load_vins()

Loaded 2855 VINs (filter_comment=None, include_openpilot=False, skipped=96)


Loaded AsBuilt data for 2855 VINs


In [3]:
import random

import pandas as pd

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.settings import VEHICLE_SETTINGS

pd.set_option('display.max_columns', None)

rows = []
for vin in ('WF0NXXGCHNJR69712', *random.sample(vins, 9)):
  abd = AsBuiltData.from_vin(vin)
  rows.append([vin, *map(lambda s: abd.get_setting_value(s), VEHICLE_SETTINGS)])

columns = [('', 'VIN'), *map(lambda s: (f'{s.ecu[0].name} ({s.ecu[1]})' if type(s.ecu) == tuple else s.ecu.name, s.comment), VEHICLE_SETTINGS)]

df = pd.DataFrame(rows, columns=pd.MultiIndex.from_tuples(columns))
df

ImageProcessingModuleA (14F403)                 \
                 VIN                         Vehicle Distance Alert   
0  WF0NXXGCHNJR69712                    C519 (Focus)            Off   
1  1FMJK2A86PEA54796                         Missing        Missing   
2  1FTFW1E81PKD73866                         Missing        Missing   
3  1FMCU9J96NUA37584             CX482 (Escape/Kuga)             On   
4  3FMCR9B65NRD90637     CX430 (Bronco Sport OnRoad)             On   
5  2FMPK4K94LBA72510                    CD539 (Edge)             On   
6  1FTVW1EL8PWG07395                         Missing        Missing   
7  1FTBR3X84RKA07188     V363 (Transit Chassis MWB2)            Off   
8  2FMPK4J90NBA09781                    CD539 (Edge)            Off   
9  1FTEW1C84PFA02871                         Missing        Missing   

                                                                  \
  Forward Collision Warning HUD High Beam Lane Departure Warning   
0                           Off       Off                     On   
1                       Missing   Missing                Missing   
2                       Missing   Missing                Missing   
3                           Off      AHBC                     On   
4                           Off      AHBC                     On   
5                           Off      AHBC                     On   
6                       Missing   Missing                Missing   
7                           Off      AHBC                     On   
8                           Off      AHBC                     On   
9                       Missing   Missing                Missing   

                                                                 \
  Lane Keeping Aid Driver Alert System Traffic Sign Recognition   
0               On                 Off                       On   
1          Missing             Missing                  Missing   
2          Missing             Missing                  Missing   
3               On                  On                       On   
4               On                  On                       On   
5               On                  On                      Off   
6          Missing             Missing                  Missing   
7               On                  On                      Off   
8               On                  On                      Off   
9          Missing             Missing                  Missing   

                                                                             \
  Traffic Sign Recognition Mode Traffic Sign Recognition Feature LKS Switch   
0                   Camera Only                             SLIF    Network   
1                       Missing                          Missing    Missing   
2                       Missing                          Missing    Missing   
3                        Fusion                             SLIF    Network   
4                        Fusion                             SLIF    Network   
5                        Fusion                              Off    Network   
6                       Missing                          Missing    Missing   
7                        Fusion                              Off    Network   
8                        Fusion                              Off    Network   
9                       Missing                          Missing    Missing   

                                                                          \
  Pre-Collision Assist Type   Enable ACC Driving Side Region and Country   
0              Radar Fusion          Off         Left               0x25   
1                   Missing      Missing      Missing            Missing   
2                   Missing      Missing      Missing            Missing   
3              Radar Fusion  RadarFusion        Right               0x7A   
4              Radar Fusion  RadarFusion        Right               0x7A   
5              Radar Fusion  RadarFusion        Right            

In [4]:
from notebooks.ford.settings import VehicleSettings

important_settings = [
  # Q3
  VehicleSettings.ipma_vehicle,
  # VehicleSettings.ipma_driving_side,
  VehicleSettings.ipma_enable_adaptive_cruise,
  VehicleSettings.ipma_enable_traffic_jam_assist,
  VehicleSettings.ipma_shift_by_wire,
  VehicleSettings.ipma_speed_limit_sign_unit,

  # Q4
  VehicleSettings.ipma_vehicle_cfg_vehicle_type,
  VehicleSettings.ipma_vehicle_cfg_acc_type,
  VehicleSettings.ipma_module_feature_cfg_tja,
  VehicleSettings.ipma_vehicle_cfg_gear_shift_by_wire,
  VehicleSettings.ipma_market_cfg_speed_limit_sign_unit,
  # VehicleSettings.ipma_market_cfg_region,
  # VehicleSettings.ipma_market_cfg_country,
  # VehicleSettings.ipma_market_cfg_driving_side,

  VehicleSettings.ipc_display_units,

  VehicleSettings.pscm_traffic_jam_assist,
  VehicleSettings.pscm_lane_centering_assist,

  VehicleSettings.abs_cruise_control_mode,
  VehicleSettings.abs_steering_gear,

  VehicleSettings.apim_country_code_first_letter,
  VehicleSettings.apim_country_code_second_letter,
  VehicleSettings.apim_lane_change_assist,
  VehicleSettings.apim_adaptive_cruise_mode,
  VehicleSettings.apim_transmission_type,
  VehicleSettings.apim_fuel_type,
  VehicleSettings.apim_wheel_base,
  VehicleSettings.apim_wheel_base_cont,
  VehicleSettings.apim_vehicle_weight,
]

rows = []
for vin in ('WF0NXXGCHNJR69712', 'WF0NXXGCHNKD75963', 'RHAETAGEXN2000154', *random.sample(vins, 19)):
  abd = AsBuiltData.from_vin(vin)
  rows.append([vin, *map(lambda s: abd.get_setting_value(s), important_settings)])

columns = [('', 'VIN'), *map(lambda s: (f'{s.ecu[0].name} ({s.ecu[1]})' if type(s.ecu) == tuple else s.ecu.name, s.comment), important_settings)]

df = pd.DataFrame(rows, columns=pd.MultiIndex.from_tuples(columns))
df

Unknown ECU address: 1808 (0x710)
0x710 {61706: 'MB3C-14E098-BD', 61712: 'DSMB3C-14E090-AF', 61713: 'MB3C-14E099-AD', 61715: 'MB3C-14E090-AF', 61832: 'MB3C-14E093-AG', 61836: '0000000001879289'}
Unknown ECU address: 1794 (0x702)
0x702 {61712: 'DSMU5T-14F548-BC', 61713: 'MU5T-14G009-JC', 61715: 'MU5T-14F548-JE', 61832: 'MU5T-14G003-JD', 61836: '1599'}
Unknown ECU address: 1891 (0x763)
0x763 {61712: 'DSMU5T-14F549-BC', 61713: 'MU5T-14G000-JC', 61715: 'MU5T-14F549-JE', 61832: 'MU5T-14F684-JD', 61836: '3978'}
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU address: 1794 (0x702)
Unknown ECU a

ImageProcessingModuleA (14F403)                          \
                  VIN                         Vehicle   Enable ACC Enable TJA   
0   WF0NXXGCHNJR69712                    C519 (Focus)          Off   Disabled   
1   WF0NXXGCHNKD75963                    C519 (Focus)          Off   Disabled   
2   RHAETAGEXN2000154             CX482 (Escape/Kuga)  RadarFusion    Enabled   
3   3FMCR9B69MRA01593     CX430 (Bronco Sport OnRoad)          Off   Disabled   
4   1FMEE5DH3MLA61072                   U725 (Bronco)          Off   Disabled   
5   2FMPK4J9XMBA44195                    CD539 (Edge)  RadarFusion    Enabled   
6   5LMJJ3TG1REL01462                         Missing      Missing    Missing   
7   1FTFW1E53MFA45536                         Missing      Missing    Missing   
8   3FMTK2SU9NMA47479                         Missing      Missing    Missing   
9   1FM5K8FW5MNA11574                 U625 (Explorer)  RadarFusion    Enabled   
10  1FT6W1EV0NWG00914                         Missing      Missing    Missing   
11  1FA6P8CF3R5416974                         Missing      Missing    Missing   
12  3FTTW8C91RRA02003                 P758 (Maverick)          Off   Disabled   
13  1FTFW1CD8NFA42031                         Missing      Missing    Missing   
14  3FTTW8H30RRA15636                 P758 (Maverick)          Off   Disabled   
15  3FMTK3SU7PMA87875                         Missing      Missing    Missing   
16  5LMCJ1DA4PUL10800                         Missing      Missing    Missing   
17  3FTTW8A33RRA16483                 P758 (Maverick)          Off   Disabled   
18  1FADP3F2XHL251518                         Missing      Missing    Missing   
19  3FMCR9B64PRD41593     CX430 (Bronco Sport OnRoad)          Off   Disabled   
20  1FMCU0MZ3PUB17435                         Missing      Missing    Missing   
21  1FMSK8DH9MGA29667                 U625 (Explorer)  RadarFusion    Enabled   

                                       ImageProcessingModuleA (14H107)  \
   Shift by Wire Speed Limit Sign Unit          VehicleCfg_VehicleType   
0            Off                   MPH                         Missing   
1            Off                   KPH                         Missing   
2             On                   KPH                         Missing   
3             On                   MPH                         Missing   
4            Off                   MPH                         Missing   
5             On                   MPH                         Missing   
6        Missing               Missing                            0x0D   
7        Missing               Missing                            0x02   
8        Missing               Missing                            0x00   
9             On                   MPH                         Missing   
10       Missing               Missing                            0x06   
11       Missing               Missing                            0x1C   
12            On                   MPH                         Missing   
13       Missing               Missing                            0x02   
14            On                   MPH                         Missing   
15       Missing               Missing                            0x08   
16       Missing               Missing                            0x16   
17            On                   MPH                         Missing   
18       Missing               Missing                         Missing   
19            On                   MPH                         Missing   
20       Missing               Missing                            0x15   
21            On                   MPH                         Missing   

                                                                       \
   VehicleCfg_ACCType ModuleFeatureCfg_TJA VehicleCfg_GearShiftByWire   
0             Missing              Missing                    Missing   
1             Missing              Missing                    Missing   
2       